In [ ]:
%load_ext nb_mypy
%nb_mypy Off

In [ ]:
from __future__ import annotations
import numpy as np
import random
import copy

from typing import Tuple, List
from numpy import array, zeros

# from Big_Class import Big_Class  # already imported one NETfuncs is imported
from User_Variables import User_Variables  # already imported one NETfuncs is imported
from Network_Structure import Network_Structure  # already imported one NETfuncs is imported
from Big_Class import Big_Class
from Network_State import Network_State
from Networkx_Net import Networkx_Net
import NETfuncs, matrix_functions, functions, statistics, plot_functions, solve

In [ ]:
## Parameters

Nin: int = 2
Nout: int = 3

# learning rate
# alpha_vec = np.array([0.2, 0.5, 1.0, 2.0, 2.5, 2.7, 3.5, 4.0, 8.0])  # 1D array = const alpha, array of arrays = annealed
# alpha_vec = np.array([np.append(np.linspace(4.1,4.8,200), np.linspace(4.8,4.1,200))])  # 1D array for constant alpha, array of arrays for annealed alpha
# alpha_vec: np.ndarray = np.array([0.2])  # for allostery, regression and general_reg
# alpha_vec = np.array([0.1])  # for general_reg_allRsChange
# alpha_vec = np.array([np.linspace(5,3.85,40)])  # 1D array for constant alpha, array of arrays for annealed alpha
# alpha_vec = np.array([4.8])
alpha: float = 1.0  # for network combine attempt

# resistance-pressure proportionality factor
gamma: np.ndarray = np.array([1.0])

# R_update: str = 'propto'
R_update = 'deltaR'

# training length
iterations = 180
    
supress_prints: bool = False
bc_noise: float = 0.0
use_p_tag: bool = True
    
M_values: np.ndarray = array([0.4, 0.25, 0.1, 0.15, 0.02, 0.3, 0.35, 0.15])
    
## Networkx sizes
scale: float = 5.0
squish: float = 0.01

In [ ]:
## Assign input and output nodes a.f.o lattice size and row choice
input_nodes_arr, ground_nodes_arr, output_nodes_arr = matrix_functions.build_input_output_and_ground(Nin, Nout)

In [ ]:
print('input_nodes_arr ', input_nodes_arr)
print('output_nodes_arr ', output_nodes_arr)
print('ground_nodes_arr ', ground_nodes_arr)

In [ ]:
## Variables class - mostly user choices
Variabs = User_Variables(iterations,\
                         Nin, \
                         Nout, \
                         gamma, \
                         R_update, \
                         use_p_tag, \
                         supress_prints, \
                         bc_noise)
Variabs.assign_alpha_vec(alpha)
print('alpha_vec', Variabs.alpha_vec)
Variabs.create_M(M_values)

In [ ]:
## Big Class containing all classes in Network Simulation
BigClass = Big_Class(Variabs)

In [ ]:
## Structure class - build incidence matrices and 1d arrays of edges

Strctr = Network_Structure(input_nodes_arr, \
                           output_nodes_arr, \
                           ground_nodes_arr,)
Strctr.build_incidence()
Strctr.build_edges()
BigClass.add_Strctr(Strctr)  # add to big class

In [ ]:
## Initiate internal flow network state class

State = Network_State(Nin, Nout)
State.initiate_resistances(BigClass)
BigClass.add_State(State)  # add to big class

In [ ]:
## build network graphics class and plot structure

NET = Networkx_Net(scale, squish)
NET.buildNetwork(BigClass)
NET.build_pos_lattice(BigClass, plot=True, node_labels=True)
BigClass.add_NET(NET)  # add to big class

## Start loop from here

In [ ]:
for i in range(Variabs.iterations):
    print('i', i)
    State.draw_p_in_and_desired(Variabs)
    print('p_drawn ', State.input_drawn)
    print('desired output', State.desired)
    State.solve_flow_until_conv(BigClass, "measure")  # measure and don't change resistances
    print('output meausred', State.output)
    if not i % 2:  # even iterations, take another sampled pressure and measure again
        pass
    else:  # odd iterations, go to dual problem and update resistances
        State.calc_loss(BigClass)
        State.update_pressure_dual(BigClass)
        State.update_output_dual(BigClass)
        State.solve_flow_until_conv(BigClass, "dual")  # measure and don't change resistances
        State.update_Rs(BigClass)

In [ ]:
plot_functions.plot_importants(State, Variabs, State.desired_in_t, Variabs.M)